In [1]:
import re
import json
import requests

In [2]:
start = '1587427200'
end = '1451520000'

url = f'https://web-api.coinmarketcap.com/v1/cryptocurrency/ohlcv/historical?convert=USD&slug=siacoin&time_end={start}&time_start={end}'

In [3]:
def make_request(url):
    response = requests.get(url)
    
    assert response.status_code == 200
    return json.loads(response.content)

In [4]:
json_response = make_request(url)

In [5]:
from collections import defaultdict

In [6]:
by_month = defaultdict(list)

quotes = json_response['data']['quotes']
for quote in quotes:
    close_time = quote['time_close']
    
    groups = re.search(r'^(\d+)-(\d+)-', close_time)
    
    year = groups.group(1)
    month = groups.group(2)
    
    key = f'{year}-{month}'.replace('-0', '-')
    by_month[key].append(
        quote['quote']['USD']['high']
    )

In [7]:
import numpy as np

In [8]:
max_high_by_month = {}

for key in by_month.keys():
    max_high_by_month[key] = np.max(by_month[key])

In [9]:
max_high_by_month

{'2016-1': 3.540329998941161e-05,
 '2016-2': 0.00014594300591852516,
 '2016-3': 0.00017359199409838766,
 '2016-4': 0.00014734899741597474,
 '2016-5': 0.00035821201163344085,
 '2016-6': 0.0011316200252622366,
 '2016-7': 0.0008670879760757089,
 '2016-8': 0.0006357270176522434,
 '2016-9': 0.0007552350289188325,
 '2016-10': 0.0004909790004603565,
 '2016-11': 0.00038899300852790475,
 '2016-12': 0.000268881005467847,
 '2017-1': 0.00041142699774354696,
 '2017-2': 0.00042406300781294703,
 '2017-3': 0.000786992022767663,
 '2017-4': 0.001294099958613515,
 '2017-5': 0.012221000157296658,
 '2017-6': 0.022363899275660515,
 '2017-7': 0.014814499765634537,
 '2017-8': 0.010056599974632263,
 '2017-9': 0.009494190104305744,
 '2017-10': 0.005499720107764006,
 '2017-11': 0.006964200176298618,
 '2017-12': 0.038732901215553284,
 '2018-1': 0.11170800030231476,
 '2018-2': 0.03489600121974945,
 '2018-3': 0.023155199363827705,
 '2018-4': 0.03268929943442345,
 '2018-5': 0.0291811004281044,
 '2018-6': 0.018454499

In [10]:
with open('../data/reddit/siacoin_price.json', 'w') as output:
    output.write(json.dumps(max_high_by_month))